In [17]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time

cuda = torch.cuda.is_available()

In [10]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/VAE/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args):
        self.file_name = file_name_pattern.format(*file_name_args)
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        
    def __getitem__(self, index):
        return (self.data[index].astype(float), 0)

    def __len__(self):
        return self.data.shape[0]
    
    def save_processed_data(self, data, loss):
        name = PROCESSED_DATA_DIR + self.file_name+"_{}.csv".format(loss)
        np.savetxt(name, data, delimiter=",")

In [29]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

CUDA = True
SEED = 1
BATCH_SIZE = 1000
LOG_INTERVAL = 10
EPOCHS = 10

ZDIMS = 4 # latent dimensions
INTERMEDIATE_DIMS = 128
FEATURES = 10

torch.manual_seed(SEED)
if CUDA:
    torch.cuda.manual_seed(SEED)

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.relu = nn.ReLU()
        
        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS)
        self.dense2_1 = nn.Linear(INTERMEDIATE_DIMS, ZDIMS)  # mu layer
        self.dense2_2 = nn.Linear(INTERMEDIATE_DIMS, ZDIMS)  # logvariance layer
        
        # this last layer bottlenecks through ZDIMS connections

        # DECODER LAYERS
        self.dense3 = nn.Linear(ZDIMS, INTERMEDIATE_DIMS)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS, FEATURES)
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.dense1(x))
        return self.dense2_1(h1), self.dense2_2(h1) #mu, logvar

    def reparameterize(self, mu, logvar):
        # If we sampled directly from the latent distribution
        # we wouldn't be able to backprop the results because
        # there is no clear grad on the distribution
        
        # This reparam samples from a unit gaussian and then scales
        # by the latent parameters giving a defined route to backprop.

        std = logvar.mul(0.5).exp_() 

        # Sample from a unit gaussian with dimensions matching
        # the latent space.
        eps = Variable(std.data.new(std.size()).normal_())

        return eps.mul(std).add_(mu) # rescale and return

    def decode(self, z):
        h3 = self.relu(self.dense3(z))
        return self.sigmoid(self.dense4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, FEATURES))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_function(recon_x, x, mu, logvar):
    
    # Cross entropy: how good is the reconstruction in terms of
    # max likelihood
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, FEATURES))

    # KLD is Kullback–Leibler divergence. Regularize VAE by
    # penalizing divergence from the prior

    # See Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    KLD /= BATCH_SIZE * FEATURES # Normalise by same number of elements as in reconstruction

    return BCE + KLD

def train(model, epoch, data_loader):
    model.train()
    train_loss = 0

    for batch_idx, (data, _) in enumerate(data_loader):
        data = Variable(data)
        data = data.float()
        if CUDA:
            data = data.cuda()
        
        optimizer.zero_grad()

        reconstructed, mu, logvar = model(data)
        
        # calculate loss
        loss = loss_function(reconstructed, data, mu, logvar)
        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
#         if batch_idx % LOG_INTERVAL == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader),
#                 loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.8f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

### Train and Process Utils

In [32]:
def train_model(model_class, dataset, dataset_number, verbose=True):
    model = model_class()
    if cuda:
        model = model.cuda()

    num_epochs = 100
    batch_size = 1000
    learning_rate = 1e-2
    lr_sched = False
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#     scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2000, 5000], gamma=0.1)

    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        if lr_sched:
            scheduler.step()

        train(model, epoch, data_loader)
    

    torch.save(model.state_dict(), "../Models/VAE_{}.pth".format(dataset_number))
    return model

def encode_data(model, dataset):
    all_data = torch.from_numpy(dataset.data)
    all_data = Variable(all_data)
    all_data = all_data.float()
    
    if cuda:
        all_data = all_data.cuda()

    output_mu, _ = model.encode(all_data)
    
    if cuda:
        output = output.cpu()
        
    dataset.save_processed_data(output.data.numpy())
    return output.data.numpy()

In [33]:
dataset = CovariateDataset("n_{}_model_{}_v_{}_covar_data", [1000, "A_add_lin", 1])
trained_model = train_model(
                                    VAE,
                                    dataset,
                                    1,
                                    verbose=True)

# encode_data(trained_model, dataset, "sparsity")

====> Epoch: 0 Average loss: 0.00001212
====> Epoch: 1 Average loss: 0.00001218
====> Epoch: 2 Average loss: 0.00001219
====> Epoch: 3 Average loss: 0.00001217
====> Epoch: 4 Average loss: 0.00001213
====> Epoch: 5 Average loss: 0.00001218
====> Epoch: 6 Average loss: 0.00001216
====> Epoch: 7 Average loss: 0.00001218
====> Epoch: 8 Average loss: 0.00001217
====> Epoch: 9 Average loss: 0.00001219
====> Epoch: 10 Average loss: 0.00001216
====> Epoch: 11 Average loss: 0.00001214
====> Epoch: 12 Average loss: 0.00001216
====> Epoch: 13 Average loss: 0.00001216
====> Epoch: 14 Average loss: 0.00001219
====> Epoch: 15 Average loss: 0.00001212
====> Epoch: 16 Average loss: 0.00001214
====> Epoch: 17 Average loss: 0.00001218
====> Epoch: 18 Average loss: 0.00001221
====> Epoch: 19 Average loss: 0.00001217
====> Epoch: 20 Average loss: 0.00001207
====> Epoch: 21 Average loss: 0.00001214
====> Epoch: 22 Average loss: 0.00001221
====> Epoch: 23 Average loss: 0.00001215
====> Epoch: 24 Average lo

In [28]:
models_to_rerun = []
num_datasets_to_process = 50
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

for dataset_number in range(num_datasets_to_process):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("-- Running for model name: ", model_name)
        
        for loss_type in loss_functions:
            print("---- Running for loss: ", loss_type)
            
            start = time()
            
            dataset = CovariateDataset("n_{}_model_{}_v_{}_covar_data", [1000, model_name, dataset_number])
            trained_model, final_loss = train_model(
                                                autoencoder,
                                                dataset,
                                                loss=loss_type,
                                                verbose=True)
            encode_data(trained_model, dataset, loss=loss_type)
            
            print("---- Done in ", time() - start, " seconds\n")
            
            # Catch bad runs
            if loss_type == loss_functions[0] and final_loss > 0.30:
                models_to_rerun.append((model_name, dataset_number, loss_type))
            elif loss_type == loss_functions[1] and final_loss > 1.0:
                models_to_rerun.append((model_name, dataset_number, loss_type))
                
    print("================\n\n")
    
print("Rerun: ", models_to_rerun)

Starting run for Dataset 0
-- Running for model name:  A_add_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1585
epoch [2000/10000], loss:0.1530
epoch [3000/10000], loss:0.1405
epoch [4000/10000], loss:0.1383
epoch [5000/10000], loss:0.1357
epoch [6000/10000], loss:0.1351
epoch [7000/10000], loss:0.1348
epoch [8000/10000], loss:0.1343
epoch [9000/10000], loss:0.1332
epoch [10000/10000], loss:0.1323
Final loss: loss:0.1323
---- Done in  69.29882407188416  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5508
epoch [2000/10000], loss:0.5670
epoch [3000/10000], loss:0.5245
epoch [4000/10000], loss:0.5220
epoch [5000/10000], loss:0.5235
epoch [6000/10000], loss:0.5195
epoch [7000/10000], loss:0.5193
epoch [8000/10000], loss:0.5189
epoch [9000/10000], loss:0.5180
epoch [10000/10000], loss:0.4298
Final loss: loss:0.4298
---- Done in  73.30059051513672  seconds

-- Running for model name:  B_add_mild_nlin
---- Running for loss:  reconstruction
epoch 

epoch [3000/10000], loss:0.6466
epoch [4000/10000], loss:0.6466
epoch [5000/10000], loss:0.6466
epoch [6000/10000], loss:0.6466
epoch [7000/10000], loss:0.6466
epoch [8000/10000], loss:0.6466
epoch [9000/10000], loss:0.6466
epoch [10000/10000], loss:0.4128
Final loss: loss:0.4128
---- Done in  69.32302975654602  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.4593
epoch [2000/10000], loss:0.4984
epoch [3000/10000], loss:0.4381
epoch [4000/10000], loss:0.4323
epoch [5000/10000], loss:0.4295
epoch [6000/10000], loss:0.4284
epoch [7000/10000], loss:0.4279
epoch [8000/10000], loss:0.3368
epoch [9000/10000], loss:0.3331
epoch [10000/10000], loss:0.3313
Final loss: loss:0.3313
---- Done in  72.52791929244995  seconds

-- Running for model name:  D_mild_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1922
epoch [2000/10000], loss:0.1680
epoch [3000/10000], loss:0.1505
epoch [4000/10000], loss:0.1472
epoch [5000/10000], loss:0.1441
epoch [6000/10

epoch [8000/10000], loss:0.1383
epoch [9000/10000], loss:0.1366
epoch [10000/10000], loss:0.1347
Final loss: loss:0.1347
---- Done in  72.66905403137207  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.3399
epoch [2000/10000], loss:0.3936
epoch [3000/10000], loss:0.3392
epoch [4000/10000], loss:0.3311
epoch [5000/10000], loss:0.3269
epoch [6000/10000], loss:0.2530
epoch [7000/10000], loss:0.2410
epoch [8000/10000], loss:0.2364
epoch [9000/10000], loss:0.2331
epoch [10000/10000], loss:0.2305
Final loss: loss:0.2305
---- Done in  75.45548033714294  seconds

-- Running for model name:  F_mod_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1737
epoch [2000/10000], loss:0.1627
epoch [3000/10000], loss:0.1467
epoch [4000/10000], loss:0.1421
epoch [5000/10000], loss:0.1386
epoch [6000/10000], loss:0.1372
epoch [7000/10000], loss:0.1364
epoch [8000/10000], loss:0.1354
epoch [9000/10000], loss:0.1339
epoch [10000/10000], loss:0.1318
Final loss: lo

epoch [1000/10000], loss:0.4803
epoch [2000/10000], loss:0.4662
epoch [3000/10000], loss:0.4242
epoch [4000/10000], loss:0.4190
epoch [5000/10000], loss:0.4172
epoch [6000/10000], loss:0.4153
epoch [7000/10000], loss:0.4148
epoch [8000/10000], loss:0.4142
epoch [9000/10000], loss:0.4135
epoch [10000/10000], loss:0.4129
Final loss: loss:0.4129
---- Done in  77.13734698295593  seconds



Starting run for Dataset 4
-- Running for model name:  A_add_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1861
epoch [2000/10000], loss:0.1591
epoch [3000/10000], loss:0.1523
epoch [4000/10000], loss:0.1495
epoch [5000/10000], loss:0.1474
epoch [6000/10000], loss:0.1463
epoch [7000/10000], loss:0.1458
epoch [8000/10000], loss:0.1452
epoch [9000/10000], loss:0.1443
epoch [10000/10000], loss:0.1436
Final loss: loss:0.1436
---- Done in  71.04707264900208  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.6494
epoch [2000/10000], loss:0.6505
epoch [3000/10000], los

epoch [4000/10000], loss:0.5292
epoch [5000/10000], loss:0.5278
epoch [6000/10000], loss:0.5266
epoch [7000/10000], loss:0.5264
epoch [8000/10000], loss:0.5261
epoch [9000/10000], loss:0.5257
epoch [10000/10000], loss:0.5253
Final loss: loss:0.5253
---- Done in  70.30084776878357  seconds

-- Running for model name:  C_add_mod_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1763
epoch [2000/10000], loss:0.1723
epoch [3000/10000], loss:0.1510
epoch [4000/10000], loss:0.1474
epoch [5000/10000], loss:0.1432
epoch [6000/10000], loss:0.1425
epoch [7000/10000], loss:0.1416
epoch [8000/10000], loss:0.1406
epoch [9000/10000], loss:0.1390
epoch [10000/10000], loss:0.1372
Final loss: loss:0.1372
---- Done in  67.50140714645386  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5766
epoch [2000/10000], loss:0.5594
epoch [3000/10000], loss:0.5296
epoch [4000/10000], loss:0.5277
epoch [5000/10000], loss:0.5266
epoch [6000/10000], loss:0.5257
epoch [7000/100

...

epoch [6000/10000], loss:0.1385
epoch [7000/10000], loss:0.1378
epoch [8000/10000], loss:0.1369
epoch [9000/10000], loss:0.1355
epoch [10000/10000], loss:0.1340
Final loss: loss:0.1340
---- Done in  68.81371593475342  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.3727
epoch [2000/10000], loss:0.3528
epoch [3000/10000], loss:0.3207
epoch [4000/10000], loss:0.2445
epoch [5000/10000], loss:0.2367
epoch [6000/10000], loss:0.2351
epoch [7000/10000], loss:0.2342
epoch [8000/10000], loss:0.2329
epoch [9000/10000], loss:0.2313
epoch [10000/10000], loss:0.2299
Final loss: loss:0.2299
---- Done in  71.80710768699646  seconds



Starting run for Dataset 44
-- Running for model name:  A_add_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.2086
epoch [2000/10000], loss:0.1962
epoch [3000/10000], loss:0.1902
epoch [4000/10000], loss:0.1837
epoch [5000/10000], loss:0.1786
epoch [6000/10000], loss:0.1771
epoch [7000/10000], loss:0.1766
epoch [8000/10000], lo

epoch [9000/10000], loss:0.1550
epoch [10000/10000], loss:0.1539
Final loss: loss:0.1539
---- Done in  68.130211353302  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.3530
epoch [2000/10000], loss:0.5373
epoch [3000/10000], loss:0.5236
epoch [4000/10000], loss:0.5214
epoch [5000/10000], loss:0.5206
epoch [6000/10000], loss:0.5194
epoch [7000/10000], loss:0.5192
epoch [8000/10000], loss:0.5190
epoch [9000/10000], loss:0.5185
epoch [10000/10000], loss:0.5183
Final loss: loss:0.5183
---- Done in  71.22183966636658  seconds

-- Running for model name:  C_add_mod_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1688
epoch [2000/10000], loss:0.1596
epoch [3000/10000], loss:0.1493
epoch [4000/10000], loss:0.1465
epoch [5000/10000], loss:0.1427
epoch [6000/10000], loss:0.1420
epoch [7000/10000], loss:0.1413
epoch [8000/10000], loss:0.1398
epoch [9000/10000], loss:0.1374
epoch [10000/10000], loss:0.1348
Final loss: loss:0.1348
---- Done in  68.4451391

epoch [1000/10000], loss:0.4609
epoch [2000/10000], loss:0.4803
epoch [3000/10000], loss:0.4321
epoch [4000/10000], loss:0.4297
epoch [5000/10000], loss:0.3336
epoch [6000/10000], loss:0.3310
epoch [7000/10000], loss:0.3289
epoch [8000/10000], loss:0.3269
epoch [9000/10000], loss:0.3251
epoch [10000/10000], loss:0.3237
Final loss: loss:0.3237
---- Done in  72.49034070968628  seconds

-- Running for model name:  E_mild_nadd_mild_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1655
epoch [2000/10000], loss:0.1590
epoch [3000/10000], loss:0.1456
epoch [4000/10000], loss:0.1425
epoch [5000/10000], loss:0.1390
epoch [6000/10000], loss:0.1381
epoch [7000/10000], loss:0.1375
epoch [8000/10000], loss:0.1365
epoch [9000/10000], loss:0.1353
epoch [10000/10000], loss:0.1336
Final loss: loss:0.1336
---- Done in  69.34417009353638  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.4693
epoch [2000/10000], loss:0.6626
epoch [3000/10000], loss:0.6485
epoch [4

epoch [6000/10000], loss:0.5359
epoch [7000/10000], loss:0.5357
epoch [8000/10000], loss:0.5355
epoch [9000/10000], loss:0.5352
epoch [10000/10000], loss:0.5349
Final loss: loss:0.5349
---- Done in  72.09613013267517  seconds

-- Running for model name:  G_mod_nadd_mod_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1944
epoch [2000/10000], loss:0.1772
epoch [3000/10000], loss:0.1662
epoch [4000/10000], loss:0.1624
epoch [5000/10000], loss:0.1597
epoch [6000/10000], loss:0.1593
epoch [7000/10000], loss:0.1589
epoch [8000/10000], loss:0.1584
epoch [9000/10000], loss:0.1579
epoch [10000/10000], loss:0.1562
Final loss: loss:0.1562
---- Done in  69.57613921165466  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5857
epoch [2000/10000], loss:0.4739
epoch [3000/10000], loss:0.4274
epoch [4000/10000], loss:0.4222
epoch [5000/10000], loss:0.4197
epoch [6000/10000], loss:0.4181
epoch [7000/10000], loss:0.4177
epoch [8000/10000], loss:0.4171
epoch [900

epoch [9000/10000], loss:0.3124
epoch [10000/10000], loss:0.3117
Final loss: loss:0.3117
---- Done in  70.06705236434937  seconds

-- Running for model name:  B_add_mild_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.2120
epoch [2000/10000], loss:0.1905
epoch [3000/10000], loss:0.1758
epoch [4000/10000], loss:0.1661
epoch [5000/10000], loss:0.1585
epoch [6000/10000], loss:0.1571
epoch [7000/10000], loss:0.1562
epoch [8000/10000], loss:0.1548
epoch [9000/10000], loss:0.1528
epoch [10000/10000], loss:0.1514
Final loss: loss:0.1514
---- Done in  67.17719197273254  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5149
epoch [2000/10000], loss:0.5171
epoch [3000/10000], loss:0.5025
epoch [4000/10000], loss:0.5085
epoch [5000/10000], loss:0.4995
epoch [6000/10000], loss:0.4962
epoch [7000/10000], loss:0.4112
epoch [8000/10000], loss:0.4069
epoch [9000/10000], loss:0.4052
epoch [10000/10000], loss:0.4041
Final loss: loss:0.4041
---- Done in  70.0500

In [8]:
models_to_rerun = [('A_add_lin', 12, 'sparsity'), ('G_mod_nadd_mod_nlin', 40, 'sparsity')]

for model_name, dataset_number, loss_type in models_to_rerun:
    dataset = CovariateDataset("n_{}_model_{}_v_{}_covar_data", [1000, model_name, dataset_number])
    trained_model, final_loss = train_model(
                                        autoencoder,
                                        dataset,
                                        loss=loss_type,
                                        verbose=True)
    encode_data(trained_model, dataset, loss=loss_type)

epoch [1000/10000], loss:0.4875
epoch [2000/10000], loss:0.4456
epoch [3000/10000], loss:0.4104
epoch [4000/10000], loss:0.3195
epoch [5000/10000], loss:0.3145
epoch [6000/10000], loss:0.3127
epoch [7000/10000], loss:0.3118
epoch [8000/10000], loss:0.3107
epoch [9000/10000], loss:0.3096
epoch [10000/10000], loss:0.3084
Final loss: loss:0.3084
epoch [1000/10000], loss:0.5869
epoch [2000/10000], loss:0.5415
epoch [3000/10000], loss:0.5173
epoch [4000/10000], loss:0.4197
epoch [5000/10000], loss:0.4216
epoch [6000/10000], loss:0.4136
epoch [7000/10000], loss:0.4132
epoch [8000/10000], loss:0.4127
epoch [9000/10000], loss:0.4123
epoch [10000/10000], loss:0.4118
Final loss: loss:0.4118
